In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from pythainlp.util import normalize
from pythainlp.ulmfit.utils import ThaiTokenizer
from pythainlp.corpus import stopwords
from pythainlp.tag import pos_tag

import pandas as pd
import numpy as np
import os, json, sys
import copy, glob
import pythainlp
import tqdm
import re, string

# Text_to_tokens
- nomalize
- remove duplicate ending character
- remove url
- remove cr
- remove unwanted character
- remove stopwords
- to lowercase
- tokenize
- remove space 

In [9]:
def text_to_tokens(text):
    # nomalize
    text = normalize(text)
    
    # remove duplicate ending character
    for m in re.finditer(r'([\u0E00-\u0E7F])(\1{1,})', text):
        text = text.replace(m.group(0),m.group(1),1)
        
    # remove url
    text = re.sub(r'http\S+', '', text)
    
    # remove CR
    text = text.replace('cr', '').replace('sr', '').replace('\xa0', '')
        
    # remove unwanted character
    pattern = re.compile(r"[^\u0E00-\u0E7Fa-zA-Z ]|ๆ")
    text = pattern.sub('',text)
    
    # remove stopword
    stop_words = stopwords.words('thai') 
    for w in stop_words:
        text.replace('w','')
        
    # lowercase
    text = text.lower()
    
    # tokenization
    tokens = pythainlp.tokenize.word_tokenize(text, engine='newmm')
    
    # remove spaces
    tokens = [token.replace(' ','') for token in tokens]
    tokens = list(filter(lambda token: token != '', tokens))

    return tokens

In [13]:
text = '''สอบถามเพื่อเอาเเอพนี้ไปเป็นตัวเลือกในการทำการบ้านหน่อยค่ะ
- เวลาเราโพสเเล้วเป็นนิรนาม เเล้วพอมีคนมาตอบเป็นเเอคนิรนามเหมือนกัน เราไม่สามารถเเยกเจ้าของโพสกับคนมาตอบได้เลยใช่มั้ย ถ้าในกรณีเเบบถ้าโดนเเอบอ้างมาเป็นคนโพส  
- กดอุนจิ นี้คือไม่ชอบใช่มั้ย
Thx'''
text_to_tokens(text)


['สอบถาม',
 'เพื่อ',
 'เอา',
 'แอ',
 'พ',
 'นี้',
 'ไป',
 'เป็น',
 'ตัวเลือก',
 'ใน',
 'การ',
 'ทำการบ้าน',
 'หน่อย',
 'ค่ะ',
 'เวลา',
 'เรา',
 'โพส',
 'แล้ว',
 'เป็น',
 'ิร',
 'นาม',
 'แล้ว',
 'พอ',
 'มี',
 'คน',
 'มา',
 'ตอบ',
 'เป็น',
 'แอ',
 'ค',
 'นิรนาม',
 'เหมือนกัน',
 'เรา',
 'ไม่',
 'สามารถ',
 'แยก',
 'เจ้าของ',
 'โพส',
 'กับ',
 'คน',
 'มา',
 'ตอบ',
 'ได้',
 'เลย',
 'ใช่',
 'มั้ย',
 'ถ้า',
 'ใน',
 'กรณี',
 'แบ',
 'ถ้า',
 'โดน',
 'แอบอ้าง',
 'มา',
 'เป็น',
 'คน',
 'โพส',
 'กด',
 'อุนจิ',
 'นี้',
 'คือ',
 'ไม่',
 'ชอบ',
 'ใช่',
 'มั้ย',
 'thx']

## pos_filter_noun
- filter noun from pos to find topics

In [4]:
def pos_filter_noun(tokens):
    postags =  pos_tag(tokens, engine = 'artagger')
    filterpos = []
    noun = ['NPRP','NCMN']
    for w in postags:
        
        # filter noun
        if w[1] in noun:
            filterpos.append(w[0])
        
    return filterpos


## fine_ques_type
- find type of question sentence


In [5]:
def fine_ques_type(text):
    
    # nomalize
    text = normalize(text)
    
    # question keywords

    where = ['ที่ไหน','อยู่ไหน','ตรงไหน','ไรดี'] 
    when = ['เมื่อไหร่','เปิดไหม','กี่โมง','กี่ทุ่ม','กี่นาที','กี่วัน','กี่เดือน','กี่ปี']
    why = ['ทำไม']
    who = ['ใคร']
    whose = ['ของใคร']
    which = ['อันไหน','หรือ']
    what = ['อะไร','แบบไหน','ทำไร','ไหน']
    how = ['อย่างไร','ยังไง','ทำไง','เท่าไหร่','เท่าไร','เป็นไง','มีกี่']
    rec = ['แนะนำ','รีวิว']
    yes_no = ['ใช่ไหม','ใช่มั้ย','ได้ไหม','หรือยัง','ได้ไหม','ไปไหม','ใช่ไหม','ได้มั้ย','มั้ย','หรือเปล่า','ไหม','ได้ป่าว','ได้ปะ','ได้รึเปล่า','มั๊ย','กันยัง']
    
    ques_types = []
    
    
            
    for w in where:
        if w in text:
            ques_types.append('where')
            break
            
    for w in when:
        if w in text:
            ques_types.append('when')
            break
            
    for w in why:
        if w in text:
            ques_types.append('why')
            break
            
    for w in who:
        if w in text:
            ques_types.append('who')
            break
            
    for w in whose:
        if w in text:
            ques_types.append('whose')
            break
            
    for w in which:
        if w in text:
            ques_types.append('which')
            break
            
    for w in what:
        if w in text and 'which' not in ques_types and 'where' not in ques_types:
            ques_types.append('what')
            break
            
    for w in how:
        if w in text:
            ques_types.append('how')
            break
            
    for w in rec:
        if w in text:
            ques_types.append('rec')
            break
            
    for w in yes_no:
        if w in text and 'when' not in ques_types:
            ques_types.append('yes_no')
            break
    
    return ques_types
        

In [6]:
df = pd.read_csv('./data/question.csv')

In [11]:
def find_types(df):
    data = []
    for t in df['text']:

        types = ','.join(fine_ques_type(t))
        
        tokens = text_to_tokens(t)
        topics = ','.join(pos_filter_noun(tokens))
        
        data.append({
            'text':t,
            'types':types,
            'keywords':topics
        })

    df2 = pd.DataFrame(data)
    return df2[['text','types','keywords']]


In [12]:
find_types(df)

,text,types,keywords
0,วัดบัวขวัญตอนนี้คนเริ่มเยอะหรือยังคะ,"which,yes_no","บัว,ขวัญ,ตอ,เยอะ,คะ"
1,สรุปโคโค่เฟรชหรือโตโต้เฟรชอะ,which,"โค,โค่เฟรช,โต้,เฟรช,อะ"
2,ปกติแล้วเวลากินซีเรียลนี่ใส่นมก่อนหรือใส่ซีเรี...,which,"เวลา,ซีเรีย,ล,นม,ซีเรีย,ล,อ่ะ"
3,ง่วงแบบนี้หนูจะทำแลบยังไงไม่ให้ผิดคะ? รายงานสด...,how,"ง่วง,แบ,หนู,แลบ,ยังไง,คะ,รายงาน,สด,องค์การ,เภส..."
4,เออ อยากรู้เหมือนกัน มันโป๊หรอ หรือเพราะคนวิจา...,which,"เอ,อยากรู้,หรอ,หัวโบราณ,tucl"
5,วันนี้ขุดทองเปิดไหมคะ,when,"วัน,ี้,ทอง,ไหม,คะ"
6,ดูหนังกันมั้ย 🎬,yes_no,"หนัง,มั้ย"
7,ไม่ได้ไปร่วมกิจกรรมวันนี้จะมีผลอารายมั้ยยงะ,yes_no,"กิจ,กรม,วัน,ี้,อา,มั้ย,งะ"
8,ฝนตกแบบนี้กินข้าวกันไหม :) #หาคนกินข้าวเป็นเพื่อน,yes_no,"ฝนตก,แบ,กินข้าว,ไหม,กินข้าว,เป็นเพื่อน"
9,ถ้ากูขำกูจะบาปไหม5555,yes_no,"กู,ขำ,กู,ไหม"
